In [5]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import multiprocessing
import pickle
import os

workdir = os.path.dirname(os.getcwd())
srcdir = os.getcwd()
datadir = workdir + '/data/'
outputdir = workdir + '/output/'
    
from PF_no_para_update import *
N = 10_000
seed = 0
obs_series = pd.read_csv(datadir + 'data.csv', delimiter=',',header = None)
obs_series = np.array(obs_series)
T = obs_series.shape[1]

case = 'actual data, seed = ' + str(seed) + ', T = ' + str(T) + ', N = ' + str(N)
try: 
    casedir = outputdir + case  + '/'
    os.mkdir(casedir)
except:
    casedir = outputdir + case  + '/'

In [6]:
D_0 = obs_series[:,[0]]
Input_0 = [[D_0, seed+i] for i in range(N)]
pool = multiprocessing.Pool()
X_t_particle = pool.map(init, tqdm(Input_0))
del(Input_0)
with open(casedir + 'X_0.pkl', 'wb') as f:
    pickle.dump(X_t_particle, f)

for t in tqdm(range(T-1)):
    print(t)
    D_t_next = obs_series[:,[t+1]]
    Input = [[D_t_next, X_t_particle[i], seed+t+i] for i in range(N)]
    del(D_t_next)
    del(X_t_particle)
    pool = multiprocessing.Pool()
    Output = pool.map(recursive, Input)
    del(Input)
    X_t_next_particle = [i[0] for i in Output]
    ν_t_next_particle = [i[1] for i in Output]   
    del(Output) 
    with open(casedir + 'X_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(X_t_next_particle, f)

    w_t_next = ν_t_next_particle/np.sum(ν_t_next_particle)
    del(ν_t_next_particle)

    try:
        count_all = sp.stats.multinomial.rvs(N, w_t_next)
    except:
        for i in range(w_t_next.shape[0]):
            if w_t_next[i]>(np.sum(w_t_next[:-1]) - 1):
                w_t_next[i] = w_t_next[i] - (np.sum(w_t_next[:-1]) - 1)
                break
        count_all = sp.stats.multinomial.rvs(N, w_t_next)

    with open(casedir + 'w_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(w_t_next, f)
    del(w_t_next)
    with open(casedir + 'count_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(count_all, f)

    X_t_particle = []       
    for i in range(N):
        if count_all[i] != 0:
            for n in range(count_all[i]):
                X_t_particle.append(X_t_next_particle[i])
    del(count_all)        
    del(X_t_next_particle)

  0%|                                                                                                                                                                                                                                                 | 0/282 [00:00<?, ?it/s]

0


  0%|▊                                                                                                                                                                                                                                        | 1/282 [00:03<17:34,  3.75s/it]

1


  1%|█▋                                                                                                                                                                                                                                       | 2/282 [00:07<18:10,  3.90s/it]

2


  1%|█▋                                                                                                                                                                                                                                       | 2/282 [00:10<23:40,  5.07s/it]Process SpawnPoolWorker-44:
Process SpawnPoolWorker-45:
Process SpawnPoolWorker-47:
Process SpawnPoolWorker-41:
Process SpawnPoolWorker-42:
Process SpawnPoolWorker-46:
Process SpawnPoolWorker-43:
Process SpawnPoolWorker-48:

Traceback (most recent call last):
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 36

  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/pyplot.py", line 57, in <module>
    from matplotlib.figure import Figure, figaspect
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/figure.py", line 25, in <module>
    from matplotlib import _blocking_input, docstring, projections
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/projections/__init__.py", line 55, in <module>
    from .. import axes, docstring
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/axes/__init__.py", line 1, in <module>
    from ._subplots import *
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/axes/_subplots.py", line 3, in <module>
    from matplotlib.axes._axes import Axes
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/axes/_axes.py", line 34, in <module>
    from matplotlib.axes._base import (
  File "/Users/haominqin/opt/anaconda

  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/artist.py", line 1345, in __init__
    self.aliasd = self.get_aliases()
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/artist.py", line 1364, in get_aliases
    if not self.is_alias(func):
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/matplotlib/artist.py", line 1443, in is_alias
    ds = inspect.getdoc(o)
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/inspect.py", line 620, in getdoc
    except (AttributeError, TypeError):
KeyboardInterrupt


KeyboardInterrupt: 